## Dataset
Zadanie klasyfikacji dla zbioru 'Titanic'.
Source: https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv
Predykcja przeżycia 'Survived' na podstawie danych o pasażerach. 

## Auto ML z UI
Step by step
* Resource group -> Create
* Machine Learning -> create
* ml.azure.com
* Manage -> Compute
* create compute instance CPU Standard_DS3_v2
* create compute clusters CPU Standard_DS3_v2
* Assets -> Datasets create dataset from web
* Headers from the first file
* Automated ML -> New automated ML run
* New experiment name
* Type Classification
* Gotowe - można sprawdzić rezultaty

## Jupyter Notebook

### Azure ML Workspace

In [1]:
pip install --upgrade --upgrade-strategy eager azureml-sdk

Requirement already up-to-date: azureml-sdk in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.21.0)
Requirement already up-to-date: azureml-core~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0.post1)
Requirement already up-to-date: azureml-train-automl-client~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: azureml-dataset-runtime[fuse]~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: azureml-train~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: azureml-pipeline~=1.21.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-sdk) (1.21.0)
Requirement already up-to-date: SecretStorage in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (from azureml-core~=1.21.0->azureml-sdk)

In [2]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.21.0


In [3]:
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()

### Load data

In [4]:
aml_dataset = ws.datasets['titanic']

full_df = aml_dataset.to_pandas_dataframe()

full_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [5]:
full_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Split dataset into train and test

In [6]:
train_dataset, test_dataset = aml_dataset.random_split(0.9, seed=1)

train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

print(train_dataset_df.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   796.000000  796.000000  796.000000  635.000000  796.000000   
mean    443.673367    0.374372    2.312814   29.855118    0.518844   
std     258.717152    0.484265    0.835614   14.724893    1.102722   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     214.750000    0.000000    2.000000   20.000000    0.000000   
50%     445.500000    0.000000    3.000000   29.000000    0.000000   
75%     666.250000    1.000000    3.000000   39.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  796.000000  796.000000  
mean     0.373116   31.958856  
std      0.794782   49.798224  
min      0.000000    0.000000  
25%      0.000000    7.895800  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


### List remote AML compute targets available

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(ws)

[{
   "id": "/subscriptions/2c4a7b68-f24b-4160-8590-5bde3308718d/resourceGroups/izabela_auto/providers/Microsoft.MachineLearningServices/workspaces/izi/computes/cpu-instances0",
   "name": "cpu-instances0",
   "location": "northeurope",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "northeurope",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS2_V2",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https://cpu-instances0.northeurope.instances.azureml.ms"
         },
         {
           "displayName": "Jupyter Lab",
           "endpointUri": "https://cpu-instances0.northeurope.instances.azureml.ms/lab"
         },
         {
           "displayName": "RStudio",
           "endpointUri": "https://cpu-instances0-8787.northeurope.instances.azureml.ms"
         }


### Connect to Remote AML Compute

In [8]:
amlcompute_cluster_name = "cpu-cluster0"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     aml_remote_compute = cts[amlcompute_cluster_name]
        
print('Checking cluster status...')
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)

Found existing training cluster.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
aml_remote_compute.get_status().serialize()

{'currentNodeCount': 0,
 'targetNodeCount': 0,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'idleNodeCount': 0,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2021-01-26T19:00:13.828000+00:00',
 'errors': None,
 'creationTime': '2021-01-26T19:00:11.093479+00:00',
 'modifiedTime': '2021-01-26T19:00:26.417204+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 0,
  'maxNodeCount': 1,
  'nodeIdleTimeBeforeScaleDown': 'PT120S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_DS2_V2'}

### List and select primary metric to drive the AutoML regression problem

In [10]:
from azureml.train import automl
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

['norm_macro_recall',
 'accuracy',
 'precision_score_weighted',
 'average_precision_score_weighted',
 'AUC_weighted']

### Define AutoML Experiment settings

In [11]:
import logging
import os

from azureml.train.automl import AutoMLConfig

project_folder = './automl'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='classification',
                             primary_metric='accuracy',
                             experiment_timeout_minutes=30,                            
                             training_data=train_dataset,
                             label_column_name="Survived",
                             n_cross_validations=5,                                                   
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             )

### Define AutoML Experiment settings

In [12]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "reg-iizi-k-automl-remote-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

reg-iizi-k-automl-remote-01-26-2021-19
Running on remote.
No run_configuration provided, running on cpu-cluster0 with default configuration
Running on remote compute: cpu-cluster0
Parent Run ID: AutoML_7dc1a82c-afc8-4e53-a5cf-4c2d3516a516

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the c

Niestety tym razem również uzyskuję błąd. Modele zapisuja się w Experiments. Ale notebook jakby się z tym nie łączył.
Nie wiem czym to jest spowodowane. 
Próbowałam uruchomić to tutaj w Azure kilkakrotnie, niestety bez powodzenia.
Najlepszy model na zbiorze treningowym to: VotingEnsamble z wynikiem 0.8492, który jest bardzo dobry.

### Explore results with Widget

In [13]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Measure Parent Run Time needed for the whole AutoML process

In [14]:
import time
import datetime as dt

run_details = run.get_details()

end_time_utc_str = run_details['endTimeUtc'].split(".")[0]
start_time_utc_str = run_details['startTimeUtc'].split(".")[0]
timestamp_end = time.mktime(datetime.strptime(end_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())
timestamp_start = time.mktime(datetime.strptime(start_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())

parent_run_time = timestamp_end - timestamp_start
print('Run Timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (parent_run_time))

Run Timing: --- 2504.0 seconds needed for running the whole Remote AutoML Experiment ---


### Retrieve the 'Best Model'

In [24]:
fitted_model = run.get_output(iteration=29)

ImportError: cannot import name 'UnhashableValueInColumn'

In [15]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

ImportError: cannot import name 'UnhashableValueInColumn'

### Make predictions

### Extract X values (feature columns) from test dataset and convert to NumPi array for predicting

In [18]:
import pandas as pd

#remove Y
if 'Survived' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('Survived')

x_test_df = test_dataset_df

### Make the actual Predictions

In [19]:
# Try the best model
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

NameError: name 'fitted_model' is not defined

In [20]:
y_predictions.shape

NameError: name 'y_predictions' is not defined

### Calculate the R2 score with Test Dataset

In [21]:
from sklearn.metrics import accuracy_score

print('Accuracy:')
accuracy_score(y_test_df, y_predictions)

Accuracy:


NameError: name 'y_predictions' is not defined